In [18]:
-- Welcome to your new notebook
-- Type here in the cell editor to add code!
CREATE OR REPLACE TEMPORARY VIEW Orders
USING PARQUET
OPTIONS(path "Files/Silver/Orders/Year=*", header "true", mode "FAILFAST");

CREATE OR REPLACE TEMPORARY VIEW OrdersDetails
USING PARQUET
OPTIONS(path "Files/Silver/Order Details/*.parquet", header "true", mode "FAILFAST");

StatementMeta(, 96ad1c9f-db47-49d0-a048-e4649bf98bad, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
DROP TABLE IF EXISTS Fact_Orders;
CREATE TABLE Fact_Orders USING delta AS
SELECT od.*, o.CustomerID, o.EmployeeID, o.OrderDate, o.Freight, o.ShipName FROM OrdersDetails od LEFT JOIN Orders o ON od.OrderID = o.OrderID

StatementMeta(, eff64563-64c9-4809-b112-d9122a751781, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
-- Crear dimension Customers en Tables (metastore)
CREATE OR REPLACE TEMPORARY VIEW Dim_Customers_Temp
USING PARQUET
OPTIONS(path "Files/Silver/Customers/*.parquet", header "true", mode "FAILFAST");

DROP TABLE IF EXISTS Dim_Customers;
CREATE TABLE Dim_Customers USING delta AS
SELECT * FROM Dim_Customers_Temp

StatementMeta(, 5e933e8c-dabb-4e7a-857c-67e704f31012, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
-- Crear dimension Shipments en Tables (metastore)
CREATE OR REPLACE TEMPORARY VIEW Dim_Shipments_Temp
USING PARQUET
OPTIONS(path "Files/Silver/Shipments/*.parquet", header "true", mode "FAILFAST");

DROP TABLE IF EXISTS Dim_Ship;
CREATE TABLE Dim_Ship USING delta
AS SELECT * FROM Dim_Shipments_Temp
--OPTIONS(path "Files/Silver/Shipments/*.parquet", header "true", mode "FAILFAST");

StatementMeta(, fec0f5e5-d246-46b9-af66-0016d9c03f1b, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [16]:
-- Crear dimension Products en Tables (metastore)
CREATE OR REPLACE TEMPORARY VIEW Dim_Products_Temp
USING PARQUET
OPTIONS(path "Files/Silver/Products/*.parquet", header "true", mode "FAILFAST");

DROP TABLE IF EXISTS Dim_Products;
CREATE TABLE Dim_Products USING delta
AS SELECT * FROM Dim_Products_Temp
--OPTIONS(path "Files/Silver/Products/*.parquet", header "true", mode "FAILFAST");

StatementMeta(, 5e933e8c-dabb-4e7a-857c-67e704f31012, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
-- Crear dimension Products en Tables (metastore)
CREATE OR REPLACE TEMPORARY VIEW Dim_Employee_Temp
USING PARQUET
OPTIONS(path "Files/Silver/Employee/*.parquet", header "true", mode "FAILFAST");

DROP TABLE IF EXISTS Dim_Employee;
CREATE TABLE Dim_Employee USING delta
AS SELECT EmployeeID, LastName, FirstName, Title, TitleOfCourtesy, BirthDate, HireDate, Address, City, Region, PostalCode, Country, ReportsTo FROM Dim_Employee_Temp
--OPTIONS(path "Files/Silver/Employee/*", header "true", mode "FAILFAST");

StatementMeta(, cf2f781b-8a0b-456c-bf5b-f1694022d864, 7, Submitted, Running)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [42]:
%%pyspark
beginDate = spark.sql(f"SELECT to_date(MIN(OrderDate)) as beginDate FROM Orders").first()[0]
endDate = spark.sql(f"SELECT to_date(MAX(OrderDate)) as endDate FROM Orders").first()[0]

spark.sql(f"select explode(sequence(to_date('{beginDate}'), to_date('{endDate}'), interval 1 day)) as calendarDate").createOrReplaceTempView('dates_temp')


StatementMeta(, 96ad1c9f-db47-49d0-a048-e4649bf98bad, 47, Finished, Available)

In [43]:
/* Creating tempiew in sql but it needs hardcode or current_time for start and end
CREATE OR REPLACE TEMPORARY VIEW dates_temp
AS SELECT explode(sequence(current_time()-365, current_time(), interval 1 month)) as calendarDate
*/
SELECT * from dates_temp

StatementMeta(, 96ad1c9f-db47-49d0-a048-e4649bf98bad, 48, Finished, Available)

<Spark SQL result set with 672 rows and 1 fields>

In [44]:
DROP TABLE IF EXISTS Dim_Date;
CREATE TABLE Dim_Date USING delta
AS
select
  year(calendarDate) * 10000 + month(calendarDate) * 100 + day(calendarDate) as dateInt,
  CalendarDate,
  year(calendarDate) AS CalendarYear,
  date_format(calendarDate, 'MMMM') as CalendarMonth,
  month(calendarDate) as MonthOfYear,
  date_format(calendarDate, 'EEEE') as CalendarDay,
  dayofweek(calendarDate) AS DayOfWeek,
  weekday(calendarDate) + 1 as DayOfWeekStartMonday,
  dayofmonth(calendarDate) as DayOfMonth,
  case
    when calendarDate = last_day(calendarDate) then 'Y'
    else 'N'
  end as IsLastDayOfMonth,
  dayofyear(calendarDate) as DayOfYear,
  quarter(calendarDate) as QuarterOfYear
from
  dates_temp
order by
  calendarDate

StatementMeta(, 96ad1c9f-db47-49d0-a048-e4649bf98bad, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>